In [80]:
""" Employed bee phase function

Input
- population of solutions (food sources)

Output
- fitness values


"""

function employed_bee_phase(population, bounds_lower, bounds_upper)
    population_new = []
    
    for i in 1:size(population)[1]
        solution = population[i, :][1]
        solution_new = create_newsolution(solution, population, bounds_lower, bounds_upper)     
        append!(population_new, [solution_new])
    end
    return population_new
end	


employed_bee_phase (generic function with 1 method)

In [82]:
function initialize_population(D, bounds_lower, bounds_upper, n)
    population = []   
    for i in 1:n
        food_source = collect(rand(bounds_lower[i]:bounds_upper[i]) for i in 1:D)
        append!(population, [food_source])
    end
        
    return population
end	
bounds_lower = [-5,-5,-5,-5]
bounds_upper = [5,5,5,5]
D=4
n=9
population = initialize_population(D, bounds_lower, bounds_upper, n)

9-element Array{Any,1}:
 [1, 1, -3, 1]
 [-3, -1, 3, -4]
 [5, 0, 5, 2]
 [4, 2, 1, 4]
 [-2, 1, -5, 1]
 [2, -1, 1, -5]
 [3, -4, 3, 4]
 [2, 2, 0, 5]
 [2, 0, 5, 3]

In [83]:
function create_newsolution(solution, population, bounds_lower, bounds_upper)
    
    # select random variable to change       
    randomvar1_index = rand(1:length(solution), 1)
        
    # select partner solution to generate new solution        
    randompartner_index = rand(1:size(population)[1], 1)
    
    # select random variable in partner solution to exchange with
        
    randompartner = population[randompartner_index, :][1]
    randomvar2_index = rand(1:length(randompartner), 1)
        
    # create new food location
    phi = rand()*2-1 #random number between -1 and 1     
    global solution_new = float(deepcopy(solution))
    a = solution[randomvar1_index] 
    b = randompartner[randomvar2_index]
    solution_new[randomvar1_index] = a + phi*(a - b)
    
    # check if lower bound is violated
    if solution_new[randomvar1_index] < bounds_lower[randomvar1_index] 
        solution_new[randomvar1_index] = bounds_lower[randomvar1_index]
    end
    
    # check if upper bound is violated
    if solution_new[randomvar1_index] > bounds_upper[randomvar1_index]
        solution_new[randomvar1_index] = bounds_upper[randomvar1_index]
    end
        
    return solution_new
end	

create_newsolution (generic function with 1 method)

In [84]:
new_population = employed_bee_phase(population, bounds_lower, bounds_upper)


9-element Array{Any,1}:
 [-1.385917104498649, 1.0, -3.0, 1.0]
 [2.2932957295090226, -1.0, 3.0, -4.0]
 [5.0, 0.0, 5.0, 2.0]
 [4.0, 2.0, 1.0, 3.7580878340034145]
 [-2.0, 1.0, -5.0, 1.0010102381785977]
 [2.0, -1.0, -0.007164088671425262, -5.0]
 [3.0, -5.0, 3.0, 4.0]
 [2.0, 2.0, 0.0, -0.10482519629638798]
 [2.0, 0.0, 5.0, 3.0]